In [38]:
using LinearAlgebra
using DataFrames
using CSV
using Plots
include("structs.jl")
include("trees.jl")
include("sourceQt.jl")
include("sourceCl.jl");

In [2]:
function do_all_paulimeasurement(ms, chann, qt)
    Vectors = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1],
            [1,1,0,0],[1,0,1,0],[1,0,0,1],[0,1,1,0],[0,1,0,1],[0,0,1,1],
            [1,1,1,0],[1,1,0,1],[1,0,1,1],[0,1,1,1],[1,1,1,1]]
    States = [state_vector(i./sqrt(sum(i)), "Z") for i in Vectors];
    Measures = [pauli_proj_meas[i][j] for (i,j) in zip(ms, qt)]
    ch = channel(kron(gates[chann], Matrix(I, 2, 2)), "Z")
    n = 100000
    file = "/home/users/ciolli/Desktop/Code/Rotations/ch-$(chann)_measurement-$(ms[1])$(qt[1])_$(ms[2])$(qt[2])_$(ms[3])$(qt[3])_$(ms[4])$(qt[4]).csv"
    df = DataFrame()
    df.θ = Float64[]
    df.Initial_State = [];

    for state in States
        Qt = QT_Dynamical_model([[-1,1]],[[-1,1]],state, ch, Measures)
        Source, Target = simulation(Qt, n) 
        T = Time_Series(n, [Qt.x;Qt.x], [Qt.y;Qt.y], Target, Source, Qt)
        push!(df.TE, TE(2, 1, 1 ,T, Freq)[1].value)
        push!(df.Initial_State, state.vec)
    end
    
    CSV.write(file, df);
end

do_all_paulimeasurement (generic function with 1 method)

In [33]:
function doall_rotations(chanl,rotations, n, word)
    Vectors = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1],
            [1,1,0,0],[1,0,1,0],[1,0,0,1],[0,1,1,0],[0,1,0,1],[0,0,1,1],
            [1,1,1,0],[1,1,0,1],[1,0,1,1],[0,1,1,1],[1,1,1,1]]
    States = [state_vector(i./sqrt(sum(i)), "Z") for i in Vectors];
    df = DataFrame()
    chann = gates[chanl]
    file = "/home/users/ciolli/Desktop/Code/Rotations/A_to_B_Brot_ch-$(chanl)_$(word).csv"
    if size(chann)[1] == 4
        ch = channel(chann, "Z")
    elseif size(chann)[1] == 2
        ch = channel(kron(chann, Matrix(I, 2, 2)), "Z")
    end
    mA = pauli_proj_meas["Mz"]["A"]
    i = 0
    for state in States
        i += 1
        J = []
        for mB in rotations
            Circuit = [mA, mB, ch, mA, mB];
            Qt = QT_Dynamical_model([[mA.value[1],mA.value[2]]],[[mB.value[1],mB.value[2]]],state, Circuit)
            First, Second = simulation(Qt, n) #return series for A and B
            Target = Second
            Source = First
            T = Time_Series(n, [Qt.x;Qt.x], [Qt.y;Qt.y], Target, Source, Qt)
            push!(J, TE(2, 1, 1 ,T, Freq)[1].value)
        end
        df[:, string(Vectors[i])] = J
    end

    CSV.write(file, df);
    return
end

doall_rotations (generic function with 1 method)

In [5]:
function noisy_simulation(chanl, measurements, states, n, nsim)
    if isempty(states)
        states = [[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1],
        [1,1,0,0],[1,0,1,0],[1,0,0,1],[0,1,1,0],[0,1,0,1],[0,0,1,1],
        [1,1,1,0],[1,1,0,1],[1,0,1,1],[0,1,1,1],[1,1,1,1]]
    end
    States = [state_vector(i./sqrt(sum(i)), "Z") for i in states];
    df = DataFrame()
    chann = gates[chanl]
    file = "/home/users/ciolli/Desktop/Code/Rotations/noisy_state_swap/A_to_B_ch-$(chanl).csv"
    if size(chann)[1] == 4
        ch = channel(chann, "Z")
    elseif size(chann)[1] == 2
        ch = channel(kron(chann, Matrix(I, 2, 2)), "Z")
    end
    i=0
    for state in States
        i += 1
        J = []
        for noise in [i/n for i in 0:n]
            Circuit = [measurements[1], measurements[2], ch, measurements[1], measurements[2]]
            Qt = QT_Dynamical_model([[measurements[1].value[1],measurements[1].value[2]]],[[measurements[2].value[1],measurements[2].value[2]]],state, Circuit)
            First, Second = noisy_simulation(Qt, nsim, noise) #return series for A and B
            Source = First
            Target = Second
            T = Time_Series(n, [Qt.x;Qt.x], [Qt.y;Qt.y], Target, Source, Qt)
            push!(J, TE(2, 1, 1 ,T, Freq)[1].value)
        end
        df[:, string(states[i])] = J
    end

    CSV.write(file, df);
    return
end

noisy_simulation (generic function with 2 methods)

In [37]:
nmeas = 30

channe = "CNOT_AB"
n = 100000;
i = 0
for (key, value) in Dict_rot(nmeas, 2)
    i += 1
    doall_rotations(channe, value, n, key);
    println(i)
end

1


2


3


4


5


6


In [6]:
n = 20

channe = "Swap"
nsim = 50000;
ms = [pauli_proj_meas["Mz"]["A"], pauli_proj_meas["Mz"]["B"]]
noisy_simulation(channe, ms, [], n, nsim)

In [17]:
n = 100
chann = "CNOT_AB"
ch = channel(gates[chann], "Z")
Vec = [[1,0,q,0] for q in [i/n for i in 0:n]]
States = [state_vector(i./sqrt(sum(i)), "Z") for i in Vec];
Circuit = [pauli_proj_meas["Mz"]["A"], pauli_proj_meas["Mz"]["B"], ch, pauli_proj_meas["Mz"]["A"], pauli_proj_meas["Mz"]["B"]]
J = []
for state in States
    Qt = QT_Dynamical_model([[Circuit[1].value[1],Circuit[1].value[2]]],[[Circuit[2].value[1],Circuit[2].value[2]]],state, Circuit)
    First, Second = simulation(Qt, 500000) #return series for A and B
    Target = Second
    Source = First
    T = Time_Series(n, [Qt.x;Qt.x], [Qt.y;Qt.y], Target, Source, Qt)
    push!(J, TE(2, 1, 1 ,T, Freq)[1].value)
end
CSV.write("/home/users/ciolli/Desktop/Code/QtTEdata/Cont_state/$(chann).csv", DataFrame(TE = J));


In [24]:
α = (0,0).*(π/2)
βone = (0,0).*(π/2)
βtwo = (0,0).*(π/2)
vec = [1,0,0,1]./sqrt(2)
ρ = vec*vec'
ch = gates["CNOT_AB"]

4×4 Matrix{Int64}:
 1  0  0  0
 0  1  0  0
 0  0  0  1
 0  0  1  0

In [52]:
qtTEan(α, βone, βtwo, ρ, ch)

0.0

In [39]:
α = (0,0).*(π/2)
βone = (0,0).*(π/2)
βtwo = (0,0).*(π/2)
vec = [1,0,0,1]./sqrt(2)
ρ = vec*vec'
ch = gates["CNOT_AB"]
range = 0:0.1:1
a = [qtTEan((i,j).*(π/2),(k,l).*(π/2),(m,n).*(π/2), ρ, ch) for i in range, j in range, k in range, l in range, m in range, n in range];

0.0

In [73]:
α = (0,0).*(π/2)
βone = (1,1).*(π/2)
βtwo = (0,0).*(π/2)
vec = [1,0,0,1]./sqrt(2)
ρ = vec*vec'
ch = gates["CNOT_AB"]
projector(βone..., 1, 2)

4×4 Matrix{ComplexF64}:
 0.5+0.0im  0.0-0.5im  0.0+0.0im  0.0-0.0im
 0.0+0.5im  0.5+0.0im  0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0-0.0im  0.5+0.0im  0.0-0.5im
 0.0+0.0im  0.0+0.0im  0.0+0.5im  0.5+0.0im

In [4]:
σy.evec

2-element Vector{Vector{ComplexF64}}:
 [0.0 - 0.7071067811865475im, -0.7071067811865475 + 0.0im]
 [0.0 - 0.7071067811865475im, 0.7071067811865475 + 0.0im]

In [7]:
pauli_proj_meas["Mx"]["B"].proj[1]

4×4 Matrix{ComplexF64}:
  0.5+0.0im  -0.5+0.0im   0.0+0.0im  -0.0+0.0im
 -0.5-0.0im   0.5+0.0im  -0.0-0.0im   0.0+0.0im
  0.0+0.0im  -0.0+0.0im   0.5+0.0im  -0.5+0.0im
 -0.0-0.0im   0.0+0.0im  -0.5-0.0im   0.5+0.0im